In [1]:
import torch
import numpy as np
import torch.nn as nn
import networks
import replaybuffer
import gym
import matplotlib.pyplot as plt
import sys, os
import utils
from mj_point_clouds import PointCloudGenerator
import open3d as o3d
import time
import PointCloudsGenerator as PCG


if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash /home/as/xvfb start
    os.environ['DISPLAY'] = ':1'

KeyboardInterrupt: 

In [ ]:
def gen_PC_for_state(pc_gen):
    points = np.asarray(pc_gen.generateCroppedPointCloud().voxel_down_sample(voxel_size = 0.04).points)
    points = points[points[:, 2] > 0.26]

    if len(points) < 800:
        points = np.concatenate([points, np.zeros((800 - len(points),3))], axis=0)
    if len(points) > 800:
        points = points[:800]
    
    return points.transpose((1, 0))

In [ ]:
class PC_env_wrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(PC_env_wrapper, self).__init__(env)
        self.pc_gen = PointCloudGenerator(env.sim, min_bound=(-1., -1., -1.), max_bound=(1., 1., 1.))
        


    def observation(self, obs):
        o = gen_PC_for_state(self.pc_gen)
        return o 

In [ ]:
class PC_env_wrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(PC_env_wrapper, self).__init__(env)
        self.pc_gen = PCG.
        


    def observation(self, obs):
        o = gen_PC_for_state(self.pc_gen)
        return o 

In [ ]:
ENV_NAME = 'Ant-v3'

env = gym.make(ENV_NAME)
test_env = gym.make(ENV_NAME)

env = PC_env_wrapper(env)
test_env = PC_env_wrapper(test_env)

env.reset()
test_env.reset()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
print(env.observation_space.shape)
print(env.action_space)

cuda:0
(111,)
Box([-1. -1. -1. -1. -1. -1. -1. -1.], [1. 1. 1. 1. 1. 1. 1. 1.], (8,), float32)


In [ ]:
import time




pc_gen = PointCloudGenerator(env.sim, min_bound=(-1., -1., -1.), max_bound=(1., 1., 1.))

start = time.time()
cloud_with_normals = pc_gen.generateCroppedPointCloud()


end = time.time()

print(end-start)



pc_gen = PCG.PointCloudGenerator(env.sim.model.cam_fovy[0], image_height=500, image_width=500, device=device)






cloud_with_normals = cloud_with_normals.voxel_down_sample(voxel_size = 0.04)


# cloud_with_normals.points = cut_z(cloud_with_normals)
a = np.asarray(cloud_with_normals.points)

# world_origin_axes = o3d.geometry.TriangleMesh.create_coordinate_frame()
# o3d.visualization.draw_geometries([])

0.34291911125183105
torch.Size([500, 500, 3])
torch.Size([250000, 3])


In [ ]:
depth = env.render('depth_array')

start = time.time()
pc = pc_gen.get_PC(depth)
end = time.time()
print(end-start)

0.0006062984466552734


In [ ]:
import plotly.graph_objects as go

def draw_pc (pc_arr):

    x = np.array([pc_arr[i][0] for i in range(pc_arr.shape[0])])
    y = np.array([pc_arr[i][1] for i in range(pc_arr.shape[0])])
    z = np.array([pc_arr[i][2] for i in range(pc_arr.shape[0])])


    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(
        size=1,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
        ))])

  
    fig.show()


In [ ]:
#TODO 
#env.observation_space.shape[0] or env.observation_space.shape

hyper = {
    # 'state_dim': env.observation_space.shape[0], 
    'state_dim': (3, 800),
    'act_dim': env.action_space.shape[0],
    'device': device,
    'enc_type': 'PointCloud',
    'hid_size': 30,
    'num_filters': 32,
    'alpha': 0.1,
    'polyak': 0.995,
    'lr': 3e-4,
    'batch_size': 256,
    'gamma': 0.99,
    'n_eval_games': 3,
    'max_episod_len': 1000,
    'eval_freq': 10000,
    'total_steps': 5000000,
    'buffer_size': 40000,
    'start_steps': 10000,
    'update_freq': 1,
    'aug_flag': False,
    'aug_pad': 4,
}

print(hyper['state_dim'])
print(hyper['act_dim'])

(3, 800)
8


In [ ]:
agent = networks.SACAgent(hyper['state_dim'], hyper['act_dim'], hyper['device'], hyper['enc_type'], hyper['hid_size'], hyper['num_filters'], hyper['alpha'], hyper['polyak'], hyper['lr'], hyper['batch_size'], hyper['gamma'])

print(agent)

SACAgent(
  (policy): Actor(
    (enc): PointCloudEncoder(
      (net): Sequential(
        (0): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
        (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Conv1d(64, 30, kernel_size=(1,), stride=(1,))
        (5): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=30, out_features=30, bias=True)
        (1): ReLU()
      )
    )
    (mu): Linear(in_features=30, out_features=8, bias=True)
    (log_std): Linear(in_features=30, out_features=8, bias=True)
  )
  (Q1): Critic(
    (enc): PointCloudEncoder(
      (net): Sequential(
        (0): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track

In [ ]:
def evaluate(env, agent, n_games=1, max_episod_len=hyper['max_episod_len']):
    reward = 0
    for n in range(n_games):
        s = env.reset()
        # s = torch.Tensor([s], dtype=torch.float32, device=hyper['device'])
        for st in range(max_episod_len):
            a = agent.get_action([s], determ=True).cpu().numpy()[0]
            s, r, d, _ = env.step(a)
            reward += r
            if d:
                break
    return (reward/n_games)

In [ ]:
step = 0
steps = []
buffer = replaybuffer.ReplayBufer(hyper['buffer_size'], hyper['act_dim'], hyper['state_dim'], hyper['device'], image_pad=hyper['aug_pad'], enc_type=hyper['enc_type'], aug_flag=hyper['aug_flag'])
state = env.reset()
return_hist = []
q1_hist = []
q2_hist = []
max_mean_return = -100000

In [2]:
import tqdm
from IPython.display import clear_output
import time

ep_len = 0


with tqdm.trange(step, hyper['total_steps']+ 1) as progress_bar:
    for step in progress_bar:
        if step > hyper['start_steps']:
            action = agent.get_action([state]).cpu().numpy()[0]
        else:
            action = env.action_space.sample()
        
        if ep_len == 0:
            with torch.no_grad():
                q1_hist.append(agent.Q1(torch.as_tensor([state], dtype=torch.float32, device=agent.device), torch.as_tensor([action], dtype=torch.float32, device=agent.device)).item())
                q2_hist.append(agent.Q2(torch.as_tensor([state], dtype=torch.float32, device=agent.device), torch.as_tensor([action], dtype=torch.float32, device=agent.device)).item())


        next_state, reward, done, _ = env.step(action)
        
        ep_len += 1



        done = False if (ep_len == hyper['max_episod_len']) else done
        buffer.store(state, action, reward, next_state, done)
        state = next_state

        if done or (ep_len == hyper['max_episod_len']):
            state = env.reset()
            ep_len = 0
        
        
        if step % hyper['update_freq'] == 0:
            s, a, r, s_next, d = buffer.sample_batch(hyper['batch_size'])
            agent.update(s, a, r, s_next, d)
            

        if (step+1) % hyper['eval_freq'] == 0:
            mean_return = evaluate(test_env, agent, hyper['n_eval_games'])
#             writer.add_scalar('pendulum_swingup', mean_return, step)
            clear_output()
            steps.append(step)
            return_hist.append(mean_return)

            if mean_return > max_mean_return:
                torch.save(agent, 'best_agent1.pth')
                max_mean_return = mean_return

            figg = plt.figure(figsize=(12, 7))

            plt.subplot(2, 2, 1)
            plt.title('Eval return')
            plt.plot(steps, return_hist)
            plt.grid(True)

            plt.subplot(2, 2, 2)
            plt.title('Init state Q1')
            plt.plot(q1_hist)
            plt.grid(True)

            plt.subplot(2, 2, 3)
            plt.title('Init state Q2')
            plt.plot(q2_hist)
            plt.grid(True)
            
            figg.savefig('gr1.png')

            torch.save(agent, 'last_agent1.pth')

NameError: name 'step' is not defined

In [ ]:
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def record_video(env, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  rec = utils.VideoRecorder(video_folder, 400, 400, fps=10)
  rec.init()
  obs = env.reset()
  for _ in range(video_length):
    rec.record(env.unwrapped)
    action = agent.get_action([obs], determ=True).cpu().numpy()[0]
    obs, _, d, _ = env.step(action)
    # print(d)
    if d:
      print(d)
      obs = env.reset()
  rec.save('Ant-PC.mp4')

sh: 1: Xvfb: not found


In [ ]:
record_video(test_env, agent)

True
True
